# AI-EFFECT: Formal Verification of Neural Networks

This is the Jupyter Notebook for the tutorial on neural network verification at the Oslo workshop of AI-EFFECT.

1. We will first show the worst-case violation, empirically found over the entire dataset.
2. Next, we will show the worst-case violation over the continuous input domain.
3. Finally, we show the worst-case distance to optimality.

[Github link for verification](https://www.github.com/bagir/verification) 

`Inline code example`

In [1]:
import os

os.environ['GRB_LICENSE_FILE'] = 'C:/Users/bagir/gurobi.lic'

# Get the current working directory
parent_directory = os.path.abspath(os.path.join(os.getcwd(), "../"))
print(parent_directory)

# Define the cases
case_name = 'case39_DCOPF'
case_iter = '1'
case_path = os.path.join(parent_directory, "python_based", "test_networks", "network_modified")

# define the neural network
nn_path = os.path.join(parent_directory, "python_based", "trained_nns", case_name, case_iter)
dataset_type = 'all'

c:\Users\bagir\OneDrive - Danmarks Tekniske Universitet\Dokumenter\1) Projects\3) AI Effect\5) Verification\DC-OPF verification


## Empirical/Statistical Worst-Case

We will first empirically evaluate the worst-case violation over the entire dataset.


In [11]:
import sys
functions_path = os.path.join(parent_directory, "python_based/functions")
sys.path.append(functions_path)

from statistical_bound import run_dc_opf_evaluation

# do the empirical evaluation of the worst-case performance of the neural network
results_summary = run_dc_opf_evaluation(case_name, case_path, nn_path)

c:\Users\bagir\AppData\Local\anaconda3\envs\verify\Lib\site-packages\pandapower\create.py:5049: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  net[table] = pd.concat([net[table], dd[dd.columns[~dd.isnull().all()]]], sort=False)


Loaded case data successfully.
Neural network data loaded successfully.
The share of always active ReLUs: 22.67 %
The share of always inactive ReLUs: 34.67 %

Summary Results:
------------------------------------------------------------
MAE (%)                                      :       0.11
Avg Max Generator Violation (MW)             :       1.21
Avg Max Line Violation (MW)                  :       0.55
Avg Distance to Optimal Setpoints (%)        :       0.49
Avg Sub-Optimality (%)                       :       0.01
Worst-Case Generator Violation (MW)          :      55.07
Worst-Case Line Violation (MW)               :      50.94
Worst-Case Distance to Optimal Setpoints (%) :       7.56
Worst-Case Sub-Optimality (%)                :       0.63


## Worst-Case Over Continuous Input Domain

Now, we will analyze what the worst-case violations are over the entire input domain, and compare them with empirically found worst-case violations.



Do some bound tightening on the ReLUs to make the MILP easier to solve. 

In [12]:
from bound_tightening import DataLoader, ReluStability, ReluBoundTightening

# load dataset (test, train, all)
data_split = "all"
data_loader = DataLoader(nn_path, data_split)
nn_relu_stability = ReluStability(nn_path, 3, data_loader)

# load nn and determine relu stability
nn_tightening = ReluBoundTightening(nn_path, 3, data_loader, nn_relu_stability)

Neural network data loaded successfully.
The share of always active ReLUs: 22.67 %
The share of always inactive ReLUs: 34.67 %
Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-24
reduction:  21.170753395587894 %
reduction:  5.957202992985689 %
bound tightening completed!


In [13]:
from exact_bound import WorstCaseAnalyzer

dataset_type = 'test'
analyzer = WorstCaseAnalyzer(case_name, case_path, nn_path, dataset_type)
analyzer.run_analysis()

c:\Users\bagir\AppData\Local\anaconda3\envs\verify\Lib\site-packages\pandapower\create.py:5049: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  net[table] = pd.concat([net[table], dd[dd.columns[~dd.isnull().all()]]], sort=False)


Loaded case data successfully.
Neural network data loaded successfully.


c:\Users\bagir\OneDrive - Danmarks Tekniske Universitet\Dokumenter\1) Projects\3) AI Effect\5) Verification\DC-OPF verification\python_based\functions\exact_bound.py:55: RuntimeWarning: Chained matrix multiplications of MVars is inefficient, collect numeric terms first when building expressions
  self.model.addConstr(self.mpc.M_g @ gp.vstack((self.pg_slack, (self.pg_pred * self.mpc.pg_delta.reshape(-1, 1) / self.mpc.baseMVA)))


Solving MILP for PGMAX Violations
0.0046160449337394015
Generator 7, Mismatch in neural network prediction -- PGMAX
Solving MILP for PGMIN Violations
Solving MILP for PLINE Violations
Branch 41, With Presolve: Mismatch in worst-case violation -- PLINE (Run: 0), Violation dcpf: 0, Violation optimization: -900.0
Line 41: Issue with solving MILP PLINE. Solver Status: 9, Objective Value: -900.0, MIP Gap: inf
Branch 44, With Presolve: Mismatch in worst-case violation -- PLINE (Run: 0), Violation dcpf: 0, Violation optimization: -899.960002812
Branch 44, Without Presolve: Mismatch in worst-case violation -- PLINE (Run: 1), Violation dcpf: 0, Violation optimization: -899.960002812

Worst-Case Summary:
--------------------------------------------------
v g time                      :      3.065
v g wc                        :    157.621
v g ID                        :          8
v line time                   :     12.541
v line wc                     :    227.530
v line ID                     

## Distance to Optimal Solution

In [14]:
from optimality_gap import SubOptimalityAnalyzer

analyzer = SubOptimalityAnalyzer(case_name, case_path, nn_path, dataset_type)
analyzer.run_analysis()

c:\Users\bagir\AppData\Local\anaconda3\envs\verify\Lib\site-packages\pandapower\create.py:5049: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  net[table] = pd.concat([net[table], dd[dd.columns[~dd.isnull().all()]]], sort=False)


Loaded case data successfully.
Neural network data loaded successfully.


c:\Users\bagir\OneDrive - Danmarks Tekniske Universitet\Dokumenter\1) Projects\3) AI Effect\5) Verification\DC-OPF verification\python_based\functions\exact_bound.py:55: RuntimeWarning: Chained matrix multiplications of MVars is inefficient, collect numeric terms first when building expressions
  self.model.addConstr(self.mpc.M_g @ gp.vstack((self.pg_slack, (self.pg_pred * self.mpc.pg_delta.reshape(-1, 1) / self.mpc.baseMVA)))


this is the max distance for generator 0:  0.018453123082550538
KKT solution and rundcopf do match -- continue
this is the max distance for generator 1:  0.6167372241140243
KKT solution and rundcopf do match -- continue
this is the max distance for generator 2:  0.4250694808632653
KKT solution and rundcopf do match -- continue
this is the max distance for generator 3:  0.25860979540781137
KKT solution and rundcopf do match -- continue
this is the max distance for generator 4:  0.22106819506141315
KKT solution and rundcopf do match -- continue
this is the max distance for generator 5:  0.02861096982378769
KKT solution and rundcopf do match -- continue
this is the max distance for generator 6:  0.15753967213828016
KKT solution and rundcopf do match -- continue
this is the max distance for generator 7:  0.02614087342103477
KKT solution and rundcopf do match -- continue
this is the max distance for generator 8:  0.1579206611482067
KKT solution and rundcopf do match -- continue
this is the 